# Export EfficientDets via AutoML

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href=""><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Run in Azure ML Studio</a>
  </td>
</table>

### Step1. Build Environment


> * **Ubuntu Server** and **Python3.8** Compiler is required for this script. We also recommand that you can use conda to build an virtual environment before running jupyter notebook.
>   
>   ```bash
>   conda create --name automl python==3.8
>   source activate automl
>   ```

In [2]:
!pip install tensorflow-model-optimization==0.7.0
!pip install tensorflowjs==3.18.0
!pip install twine==1.9.0
!pip install urllib3==1.21.1
!pip install pandas==1.4.1
!pip install numpy==1.23.5
!pip install numba==0.53
!pip install tflite-model-maker==0.4.2
!pip install tf2onnx

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.20.0 requires pandas>=1.5.0, but you have pandas 1.4.1 which is incompatible.
bigframes 1.25.0 requires pandas>=1.5.3, but you have pandas 1.4.1 which is incompatible.
cudf-cu12 24.10.1 requires numba>=0.57, but you have numba 0.55.0 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.4.1 which is incompatible.
ibis-framework 9.2.0 requires pandas<3,>=1.5.3, but you have pandas 1.4.1 which is incompatible.
mizani 0.13.0

  Using cached numpy-1.21.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached numpy-1.21.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.21.6 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.21.6 which is incompatible.
arviz 0.20.0 requires numpy>=1.23.0, but you have numpy 1.21.6 which is incompatible.
astropy 6.1.4 requires numpy>=1.23, but you have numpy 1.21.6 which is incompatible.
bigframes 1.25.0 requires numpy>=1.24.0, but you have numpy 1.21.6 which is incompatible.
chex 0.1.87 requires numpy>=1

  Using cached tflite_model_maker-0.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tf_models_official-2.3.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tflite-model-maker to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 0.52.0 Requires-Python >=3.6,<3.9; 0.52.0rc3 Requires-Python >=3.6,<3.9; 0.53.0 Requires-Python >=3.6,<3.10; 0.53.0rc1.post1 Requires-Python >=3.6,<3.10; 0.53.0rc2 Requires-Python >=3.6,<3.10; 0.53.0rc3 Requires-Python >=3.6,<3.10; 0.53.1 Requires-Python >=3.6,<3.10; 0.54.0 Requires-Python >=3.7,<3.10; 0.54.0rc2 Requires-Python >=3.7,<3.10; 0.54.0rc3 Requires-Python >=3.7,<3.10; 0.54.1 Requires-Python >=3.7,<3.10
ERROR: Could not find a version that satisfies the requirement numba==0.53 (from tflite-model-maker) (from versio

### Step2. Load and Export Model Configuration

Download and extract the sample dataset for object detection, which compatible format is:
  * [Pascal_Voc](https://itriaihub.blob.core.windows.net/datasets/Detect_HardHat_244x244.pascal_voc.tar.xz)

In [2]:
!wget https://itriaihub.blob.core.windows.net/github-download-resources/repository/ITRI-AI-Hub/datasets/HardHat_Dataset.PascalVOC.zip
!unzip HardHat_Dataset.PascalVOC.zip

--2024-11-06 08:44:19--  https://itriaihub.blob.core.windows.net/github-download-resources/repository/ITRI-AI-Hub/datasets/HardHat_Dataset.PascalVOC.zip
Resolving itriaihub.blob.core.windows.net (itriaihub.blob.core.windows.net)... 20.209.42.36
Connecting to itriaihub.blob.core.windows.net (itriaihub.blob.core.windows.net)|20.209.42.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3728463 (3.6M) [application/x-zip-compressed]
Saving to: ‘HardHat_Dataset.PascalVOC.zip’

HardHat_Dataset.Pas 100%[===================>]   3.56M  1.23MB/s    in 2.9s    

2024-11-06 08:44:23 (1.23 MB/s) - ‘HardHat_Dataset.PascalVOC.zip’ saved [3728463/3728463]

Archive:  HardHat_Dataset.PascalVOC.zip
   creating: HardHat_Dataset.PascalVOC/
  inflating: HardHat_Dataset.PascalVOC/README.roboflow.txt  
   creating: HardHat_Dataset.PascalVOC/test/
  inflating: HardHat_Dataset.PascalVOC/test/000008_jpg.rf.8ddEpaD8kjf9Wwgm1P3I.jpg  
  inflating: HardHat_Dataset.PascalVOC/test/000008_jpg.

In [3]:
from tflite_model_maker import object_detector
import numpy as np

DATASET_FOLDER_PATH = '/content/HardHat_Dataset.PascalVOC'
#CLASSES = np.load(f'{DATASET_FOLDER_PATH}/data.npy').tolist()

train_data = object_detector.DataLoader.from_pascal_voc(f'{DATASET_FOLDER_PATH}/train', f'{DATASET_FOLDER_PATH}/train', CLASSES)
valid_data = object_detector.DataLoader.from_pascal_voc(f'{DATASET_FOLDER_PATH}/test', f'{DATASET_FOLDER_PATH}/test', CLASSES)

ModuleNotFoundError: No module named 'tflite_model_maker'

Build Model with Training

In [ ]:
from tflite_model_maker import model_spec
import tensorflow as tf

BACKBONE = 'efficientdet_lite4' # @param ["efficientdet_lite0", "efficientdet_lite1", "efficientdet_lite2", "efficientdet_lite3", "efficientdet_lite4"]

# Pre-train is necessary in AutoML Framework
model = object_detector.create(train_data, model_spec=model_spec.get(BACKBONE), epochs=1, batch_size=8,
                               train_whole_model=True, validation_data=valid_data)

We can preview `Params` of this model via Summary() and fill it to ITRI-AI-Hub*

In [ ]:
model.summary()

3. Export the model and upload to [ITRI Azure Instance]() and link to [Mode Zoo]().

* **Format 1**: TFLite
(*We can preview `GFLOPS` of this model via console outputs and fill it to ITRI-AI-Hub**)

In [ ]:
from tflite_model_maker.config import ExportFormat

model.export(export_dir=f'{BACKBONE}', export_format=[ExportFormat.TFLITE])

* **Format 2**: ONNX

In [ ]:
!python -m tf2onnx.convert --opset 16 --tflite {BACKBONE}/model.tflite --output {BACKBONE}/model.onnx

### Output differents after export (*not prepared yet)

### Benchmarking
> * You can request access to `model_zo_library` from your ITRI-AI-Hub collaborator and port the `model` and `model_zoo_library` on your device to implement the standardised test procedure.
>   
>   ```bash
>   git clone https://github.com/R300-AI/model_zoo_library.git
>   cd model_zoo_library
>
>   python3 profile.py --model <path-to-model>.tflite --platform <HW_series_name> --chipset <processor_type>
>   ```